In [1]:
import sys
sys.path.insert(1, '/Users/brian/crypto/crocket')

In [2]:
from datetime import datetime, timedelta, timezone
from decimal import Decimal, getcontext
from json import load as json_load
from pprint import pprint
from time import sleep
from numpy import average

from os import environ
from os.path import join

from crocket.bittrex.bittrex2 import Bittrex
from crocket.sql.sql import Database
from crocket.utilities.passcode import AESCipher
from crocket.utilities.credentials import get_credentials

getcontext().prec = 8

In [132]:
def format_time(datetime_to_format, time_format = "%Y-%m-%d %H:%M:%S.%f"):
    """
    Get time now.
    Ex: 2017-09-22 12:28:22
    :return:
    """
    return datetime_to_format.strftime(time_format)


def convert_bittrex_timestamp_to_datetime(timestamp, time_format = "%Y-%m-%dT%H:%M:%S.%f"):
    
    try:
        converted_datetime = datetime.strptime(timestamp, time_format)
    except ValueError:
        converted_datetime = datetime.strptime('{}.0'.format(timestamp), time_format)
        
    return converted_datetime


def utc_to_local(utc_dt):
    return utc_dt.replace(tzinfo=timezone.utc).astimezone(tz=None)

In [135]:
format_time(utc_to_local(convert_bittrex_timestamp_to_datetime(market_history[0].get('TimeStamp'))), "%Y-%m-%d %H:%M:%S")

'2017-11-01 21:32:19'

In [4]:
# Environment variables

CROCKET_PATH = '/Users/brian/crypto/crocket'
CROCKET_KEY_PATH = join(CROCKET_PATH, 'crocket_key.json')

HOME_DIRECTORY_PATH = environ['HOME']
CREDENTIALS_FILE_PATH = join(HOME_DIRECTORY_PATH, '.credentials.json')

BITTREX_CREDENTIALS_PATH = '/Users/brian/crypto/crocket/bittrex_credentials.json'

HOSTNAME = 'localhost'
DATABASE_NAME = 'BITTREX'

BASE_COIN = 'BTC'

In [5]:
# Load crocket key
with open(CROCKET_KEY_PATH, 'r') as f:
    key = json_load(f).get('key')
    
cipher = AESCipher(key)

# Load SQL username and password
USERNAME, PASSCODE = map(lambda x: cipher.decrypt(str.encode(x)), get_credentials(CREDENTIALS_FILE_PATH))

# Load bittrex credentials
with open(BITTREX_CREDENTIALS_PATH, 'r') as f:
    BITTREX_CREDENTIALS = json_load(f)

In [ ]:
# Initialize database
db = Database(hostname=HOSTNAME,
              username=USERNAME,
              password=PASSCODE,
              database_name=DATABASE_NAME)

In [6]:
# Initialize bittrex
bittrex = Bittrex(api_key=BITTREX_CREDENTIALS.get('key'), 
                  api_secret=BITTREX_CREDENTIALS.get('secret'), 
                  api_version='v1.1')

In [124]:
def calculate_metrics(data, start_datetime, digits = 8):
    
    metrics = {}
        
    decimal_places = Decimal(10) ** (digits * -1)
    
    volume = 0
    buy_order = 0
    sell_order = 0
    price = 0
    price_volume_weighted = 0
    formatted_time = format_time(utc_to_local(convert_bittrex_timestamp_to_datetime(start_datetime)), "%Y-%m-%d %H:%M:%S")
    
    if data and isinstance(data[0], dict):

        p, v, o = map(list, zip(*[(x.get('Price'), x.get('Total'), x.get('OrderType')) for x in data]))

        volume = sum(v)
        buy_order = sum([1 for x in o if x == 'BUY'])
        sell_order = len(o) - buy_order

        price = (sum([Decimal(x).quantize(decimal_places) for x in p]) / len(p)).quantize(decimal_places)
        price_volume_weighted = (sum(
                [Decimal(x).quantize(decimal_places) * Decimal(y) for x, y in zip(p, v)]) / Decimal(sum(v))).quantize(decimal_places)
    
    metrics = {'volume': volume,
               'buy_order': buy_order,
               'sell_order': sell_order,
               'price': price,
               'price_volume_weighted': price_volume_weighted,
               'time': formatted_time}
    
    return metrics


def get_interval_index(entries, target_datetime, interval):
    
    timestamp_list = [convert_bittrex_timestamp_to_datetime(x.get('TimeStamp')) for x in entries]
    
    stop_index = len([x for x in timestamp_list if x > target_datetime])
    start_index = len([x for x in timestamp_list if (x - target_datetime).total_seconds() > interval])
    
    return start_index, stop_index


def format_bittrex_entry(data):
    """
    Format market summary object into SQL entry.
    :param market_summary: Summary of market
    :param time: Time
    :return: (list) tuples
    """

    formatted_entry = []

    formatted_entry.append(('time', data.get('time')))
    formatted_entry.append(('price', data.get('price')))
    formatted_entry.append(('wprice', data.get('wprice')))
    formatted_entry.append(('basevolume', data.get('basevolume')))
    formatted_entry.append(('buyorder', data.get('buyorder')))
    formatted_entry.append(('sellorder', data.get('sellorder')))

    return formatted_entry

In [125]:
market = 'BTC-GRS'

sleep_time = 30 # seconds

interval = 60 # seconds

constant = 0

initial_market_history = bittrex.get_market_history(market).get('result')

working_list = initial_market_history
print('Initial working list: {}'.format(len(working_list)))
current_datetime = convert_bittrex_timestamp_to_datetime(initial_market_history[0].get('TimeStamp'))

sleep(30)

for ttt in range(0, 10):

    try:
        market_history = bittrex.get_market_history(market).get('result')
        
        pprint(market_history[0])

        last_id = working_list[0].get('Id')

        index = [x.get('Id') for x in market_history].index(last_id)

        working_list = market_history[:index] + working_list
        print('Working list: {}'.format(len(working_list)))

        latest_datetime = convert_bittrex_timestamp_to_datetime(working_list[0].get('TimeStamp'))

        if (latest_datetime - current_datetime).total_seconds() > interval:
            
            start, stop = get_interval_index(working_list, current_datetime, interval)
            metrics = calculate_metrics(working_list[start:stop], current_datetime)
            pprint(metrics)
            pprint(current_datetime)

            working_list = working_list[:start]
            print('Working list: {}'.format(len(working_list)))
            current_datetime = current_datetime + timedelta(seconds=interval)

            constant = 0

        elif constant > 2:
            
            start, stop = get_interval_index(working_list, current_datetime, interval)
            new_metrics = calculate_metrics(working_list[start_index:stop_index], current_datetime) # All metrics should be 0
            
            metrics['volume'] = new_metrics.get('volume')
            metrics['buy_order'] = new_metrics.get('buy_order')
            metrics['sell_order'] = new_metrics.get('sell_order')
            metrics['time'] = new_metrics.get('time')
            
            pprint(metrics)
            pprint(current_datetime)
            
            working_list = working_list[:start_index]
            print('Working list: {}'.format(len(working_list)))
            current_datetime = current_datetime + timedelta(seconds=interval)

        else:
            constant += 1
            print('Latest entry within interval. Adding constant.')

        # Set dynamic sleep time based on order volume
        if index < 30:
            sleep_time = 60
        else:
            sleep_time = 30

    except ValueError:
        print('Latest ID in working list not found in latest market history.')

        working_list = market_history + working_list
        
        start, stop = get_interval_index(working_list, current_datetime, interval)
        
        metrics = calculate_metrics(working_list, current_datetime)
        pprint(metrics)
        pprint(current_datetime)
        
        current_datetime = current_datetime + timedelta(seconds=interval)
        
        sleep_time = 30

    sleep(sleep_time)

Initial working list: 200
{'FillType': 'PARTIAL_FILL',
 'Id': 7052932,
 'OrderType': 'SELL',
 'Price': 0.00011298,
 'Quantity': 224.74627669,
 'TimeStamp': '2017-11-02T04:25:36.817',
 'Total': 0.02539183}
Working list: 211
Latest entry within interval. Adding constant.
{'FillType': 'FILL',
 'Id': 7053400,
 'OrderType': 'BUY',
 'Price': 0.00011737,
 'Quantity': 4.19949327,
 'TimeStamp': '2017-11-02T04:27:00.46',
 'Total': 0.00049289}
Working list: 309
{'buy_order': 29,
 'price': Decimal('0.00011311'),
 'price_volume_weighted': Decimal('0.00011297'),
 'sell_order': 6,
 'time': datetime.datetime(2017, 11, 2, 4, 25, 10, 830000),
 'volume': 3.1753889199999996}
datetime.datetime(2017, 11, 2, 4, 25, 10, 830000)
Working list: 74
{'FillType': 'PARTIAL_FILL',
 'Id': 7053546,
 'OrderType': 'SELL',
 'Price': 0.0001174,
 'Quantity': 17.4672489,
 'TimeStamp': '2017-11-02T04:27:14.537',
 'Total': 0.00205065}
Working list: 115
{'buy_order': 64,
 'price': Decimal('0.00011580'),
 'price_volume_weighted'